# PrimeurVision - Evaluation du modele YOLOv8

Evaluation du meilleur modele issu de l'entrainement sur le jeu de test.

**Metriques** : mAP@50, mAP@50-95, Precision, Recall, AP par classe

**Classes** : carotte, aubergine, citron, pomme_de_terre, radis, tomate

## 1. Installation

In [ ]:
!pip install ultralytics -q

## 2. Imports

In [ ]:
import os
import shutil
import random
import glob
import yaml
import matplotlib.pyplot as plt
from PIL import Image
from ultralytics import YOLO

## 3. Chargement du modele et du dataset

Le modele doit avoir ete sauvegarde sur Google Drive apres l'entrainement (`My Drive/PrimeurVision/models/`).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MODEL_PATH = '/content/drive/MyDrive/PrimeurVision/models/best_yolov8n_primeurvision.pt'
DATASET_SRC = '/content/drive/MyDrive/PrimeurVision/dataset'
MODELS_DIR = '/content/drive/MyDrive/PrimeurVision/models'
WORK_DIR = '/content/dataset'
CONF_THRESHOLD = 0.25

# Copie en local pour accelerer
if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)
shutil.copytree(DATASET_SRC, WORK_DIR)

# Charger la config du dataset
data_yaml_path = os.path.join(WORK_DIR, 'data.yaml')
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

# Mettre a jour les chemins locaux
data_config['path'] = WORK_DIR
data_config['train'] = 'images/train'
data_config['val'] = 'images/val'
data_config['test'] = 'images/test'
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

CLASS_NAMES = data_config['names']

# Charger le modele
model = YOLO(MODEL_PATH)
print(f"Modele charge : {MODEL_PATH}")
print(f"Classes : {CLASS_NAMES}")

n_test = len(os.listdir(os.path.join(WORK_DIR, 'images', 'test')))
print(f"Images de test : {n_test}")

## 4. Evaluation quantitative sur le test

In [ ]:
metrics = model.val(data=data_yaml_path, split='test')

print("Resultats sur le test :")
print(f"  mAP@50    : {metrics.box.map50:.4f}")
print(f"  mAP@50-95 : {metrics.box.map:.4f}")
print(f"  Precision : {metrics.box.mp:.4f}")
print(f"  Recall    : {metrics.box.mr:.4f}")

print("\nAP@50 par classe :")
for i, class_name in CLASS_NAMES.items():
    ap50 = metrics.box.ap50[i] if i < len(metrics.box.ap50) else 0
    print(f"  {class_name:20s} : {ap50:.4f}")

## 5. Matrice de confusion

In [ ]:
# La matrice de confusion est generee automatiquement par model.val()
eval_dir = str(metrics.save_dir)
confusion_img = os.path.join(eval_dir, 'confusion_matrix.png')

if os.path.exists(confusion_img):
    plt.figure(figsize=(8, 8))
    plt.imshow(Image.open(confusion_img))
    plt.axis('off')
    plt.title('Matrice de confusion (test)')
    plt.show()

# Si les courbes d'entrainement ont ete sauvegardees sur Drive
curves_img = os.path.join(MODELS_DIR, 'results.png')
if os.path.exists(curves_img):
    plt.figure(figsize=(18, 8))
    plt.imshow(Image.open(curves_img))
    plt.axis('off')
    plt.title('Courbes d\'entrainement')
    plt.show()

## 6. Resultats qualitatifs - predictions sur le test

In [ ]:
test_images = glob.glob(os.path.join(WORK_DIR, 'images', 'test', '*.jpg'))
sample_test = random.sample(test_images, min(6, len(test_images)))

n_cols = min(3, len(sample_test))
n_rows = (len(sample_test) + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 6 * n_rows))
if len(sample_test) == 1:
    axes = [axes]
else:
    axes = axes.flatten()

for ax, img_path in zip(axes, sample_test):
    result = model.predict(img_path, conf=CONF_THRESHOLD, verbose=False)[0]
    ax.imshow(result.plot()[:, :, ::-1])
    ax.set_title(os.path.basename(img_path), fontsize=8)
    ax.axis('off')

for i in range(len(sample_test), len(axes)):
    axes[i].axis('off')

plt.suptitle('Predictions sur le test', fontsize=14)
plt.tight_layout()
plt.show()